Sources  
- http://host.robots.ox.ac.uk/pascal/VOC/voc2007/ (pascal dataset)
- https://github.com/keshik6/pascal-voc-classification/tree/master (pascal github inspo)
- http://host.robots.ox.ac.uk/pascal/VOC/voc2007/devkit_doc_07-Jun-2007.pdf (pascal devkit)
- https://github.com/FabrizioDeSantis/Object-Detection-Manga109/tree/main (manga109 github inspo)
- manga109 dataset \
- manga109 libraries, api 
- manga109 paper 
- https://towardsdatascience.com/bounding-box-prediction-from-scratch-using-pytorch-a8525da51ddc 

Dependencies 

In [1]:
import manga109api_custom
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torchvision.transforms as T 
from dataset import CustomDataset
from torch.utils.data import DataLoader
import albumentations as A 
import pandas as pd
from albumentations.pytorch import ToTensorV2
import numpy as np
from matplotlib import pyplot as plt

Load Data (fix paths)

In [2]:
# directory with Manga109 data 
manga109_root_dir = "./Manga109/Manga109_released_2023_12_07" # directory with Manga109 data

# custom parser to access Manga109 data 
p = manga109api_custom.Parser(root_dir=manga109_root_dir)
img_dict = p.load_all_images()

# create pandas dataframe from the dictionary
df = pd.DataFrame(img_dict)
print(df)

                                                img_path  width  height  \
0      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
1      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
2      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
3      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
4      C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
...                                                  ...    ...     ...   
10125  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
10126  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
10127  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
10128  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   
10129  C:\Users\Grace\Documents\Manga109-BoundingDete...   1654    1170   

              book_id                                             bboxes  \
0                ARMS  

Split and Stratify Data 

In [3]:
test_ratio = 0.2
val_ratio = 0.2

# split data into training/validation and testing sets
train_val_df, test_df = train_test_split(df, test_size=test_ratio, stratify=df['book_id'], random_state=42)

# further split into training and validation sets 
train_df, val_df = train_test_split(train_val_df, test_size=val_ratio, stratify=train_val_df['book_id'], random_state=42)

print(f"Total Images: {len(df)}")
print(f"Training Images: {len(train_df)}")
print(f"Validation Images: {len(val_df)}")
print(f"Testing Images: {len(test_df)}")

Total Images: 10130
Training Images: 6483
Validation Images: 1621
Testing Images: 2026


Transforms

In [4]:
# define training transform
train_transform = A.Compose([ 
    A.CenterCrop(height=1170, width=1650, p=1),                                 # crop first                         
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    #A.ColorJitter(brightness=(0.80, 1.20)),
    #A.Rotate(limit=90,p=0.5),                 
    #A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.Resize(height=780, width=1100, interpolation=1, p=1),                    # resize last 
    ToTensorV2()                                                               # to tensor
], bbox_params=A.BboxParams(format='coco',label_fields=['labels'])) 

# define validation transform
val_transform = A.Compose([
    A.CenterCrop(height=1170, width=1650, p=1),
    A.Resize(height=780, width=1100, interpolation=1, p=1),
    ToTensorV2()
])

Create Datasets & DataLoaders 

In [5]:
# datasets
train_dataset = CustomDataset(train_df, train_transform)
val_dataset = CustomDataset(val_df, val_transform)

# dataloaders 
train_loader = DataLoader(train_dataset, batch_size=8, num_workers=4, shuffle=True)
valid_loader = DataLoader(val_dataset, batch_size=8, num_workers=4)

Visualize Data - testing & validation images  

In [6]:
image = next(iter(train_loader))

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.imshow(np.transpose(np.asarray(image[0]), (1, 2, 0)))
ax.set_title('Please Draw Boxes')
plt.show()

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 54, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\_utils\collate.py", line 277, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Grace\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\_utils\collate.py", line 144, in collate
    return [collate(samples, collate_fn_map=collate_fn_map) for sam

In [ ]:
print(type(bboxes))

Define and Train Model 

In [ ]:
model = models.resnet34
# optimizer = something 
# loss function = something 
# epochs

- Validation Metrics 
- Testing Images Visualization 
- Testing 
- Testing Evaluation Metrics 